In [6]:
import numpy as np

from PIL import Image
import colours

from matplotlib import pyplot as plt
from matplotlib import pylab as pylab
%matplotlib inline
pylab.rcParams['figure.figsize'] = (12, 12)


cspace = colours.OKLAB

In [7]:
def im_rgb_to_cspace(im_r, im_g, im_b):
    im_r = np.asarray(im_r)
    im_g = np.asarray(im_g)
    im_b = np.asarray(im_b)

    rgb = np.zeros([im_r.size, 3])

    rgb[:, 0] = im_r.ravel()
    rgb[:, 1] = im_g.ravel()
    rgb[:, 2] = im_b.ravel()

    xyz = colours.SRGB.to_xyz(rgb)
    csp = cspace.from_xyz(xyz)

    l, m, s = csp[:, 0], csp[:, 1], csp[:, 2]

    l = l - cspace.l_min
    m = m - cspace.m_min
    s = s - cspace.s_min

    l = l.reshape(im_r.shape)
    m = m.reshape(im_r.shape)
    s = s.reshape(im_r.shape)

    return (l, m, s)

def im_cspace_to_rgb(im_l, im_m, im_s):
    l = np.asarray(im_l)
    m = np.asarray(im_m)
    s = np.asarray(im_s)

    l = l + cspace.l_min
    m = m + cspace.m_min
    s = s + cspace.s_min

    csp = np.zeros([im_l.size, 3])

    csp[:, 0] = im_l.ravel()
    csp[:, 1] = im_m.ravel()
    csp[:, 2] = im_s.ravel()

    xyz = cspace.to_xyz(csp)
    rgb = colours.SRGB.from_xyz(xyz)

    r, g, b = rgb[:, 0], rgb[:, 1], rgb[:, 2]

    r = r.reshape(im_l.shape)
    g = g.reshape(im_l.shape)
    b = b.reshape(im_l.shape)

    return (r, g, b)


image = Image.open(f'../../images/earth/earth-1024.png').convert('RGB')

image = np.asarray(image)

r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]

def crop_rgb_arrays(r, g, b):
    r = r[:r.shape[0] - r.shape[0]%8, :r.shape[1] - r.shape[1]%8]
    g = g[:g.shape[0] - g.shape[0]%8, :g.shape[1] - g.shape[1]%8]
    b = b[:b.shape[0] - b.shape[0]%8, :b.shape[1] - b.shape[1]%8]

    return r, g, b

def save_rgb_arrays(r, g, b, filename):
    image_cropped = np.zeros([r.shape[0], r.shape[1], 3], dtype=np.uint8)

    image_cropped[:, :, 0] = r
    image_cropped[:, :, 1] = g
    image_cropped[:, :, 2] = b

    Image.fromarray(image_cropped).save(filename)

r, g, b = crop_rgb_arrays(r, g, b)
save_rgb_arrays(r, g, b, 'original.png')


l, m, s = im_rgb_to_cspace(r, g, b)

In [8]:
im_l = l
im_m = m
im_s = s

In [10]:
im_r, im_g, im_b = im_cspace_to_rgb(im_l, im_m, im_s)

im_r = np.clip(np.rint(im_r), 0, 255).astype(np.uint8)
im_g = np.clip(np.rint(im_g), 0, 255).astype(np.uint8)
im_b = np.clip(np.rint(im_b), 0, 255).astype(np.uint8)


img_rec = np.zeros((1024, 1016, 3), dtype=np.uint8)

img_rec[:, :, 0] = im_r
img_rec[:, :, 1] = im_g
img_rec[:, :, 2] = im_b

im_rec = Image.fromarray(img_rec)

im_rec.save('recovered.png')